In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 設定 Pandas 顯示選項 (避免欄位被 ... 隱藏)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format) # 浮點數只顯示兩位

In [31]:
import pandas as pd
import numpy as np

# 讀取 CSV (遇到亂碼時嘗試 encoding)
df = pd.read_csv('daily_challenge.csv', encoding='utf-8') 
# 常見編碼還有: 'big5' (繁體中文舊編碼), 'cp950' (Windows中文), 'utf-8-sig'

# 讀取 Excel
# df = pd.read_excel('data.xlsx', sheet_name='Sheet1')

# 📋 從剪貼簿讀取 (超實用！複製網頁/Excel表格後直接執行)
# df = pd.read_clipboard()

In [32]:
# 基本體檢
print(df.shape)          # (列數, 欄數) - 確認資料量級
print(df.columns)        # 查看所有欄位名稱
print(df.head())         # 看前 5 筆資料長相
print(df.info())         # 檢查資料型態 (Dtype) 和是否有缺值 (Non-Null)

# 數值統計 (檢查是否有怪異的極大/極小值)
print(df.describe())     

# 🕵️‍♀️ 類別偵查 (抓出 'Male', 'male ', 'M' 這種不一致)
# dropna=False 會連 NaN 一起顯示，非常重要！
print(df['Cust_ID'].value_counts(dropna=False))

(1050, 5)
Index(['Cust_ID', 'Name', 'Income', 'Join_Date', 'Credit_Score'], dtype='object')
   Cust_ID    Name   Income   Join_Date  Credit_Score
0     1.00  User_1    85093  2024/11/28        718.00
1     2.00  User_2   108773  2024-09-27        479.00
2     3.00     NaN  $120050  2024-08-31        348.00
3     4.00  User_4    33807  2024-09-12        405.00
4     5.00  User_5   112937  2024-03-11           NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Cust_ID       945 non-null    float64
 1   Name          942 non-null    object 
 2   Income        943 non-null    object 
 3   Join_Date     945 non-null    object 
 4   Credit_Score  946 non-null    float64
dtypes: float64(2), object(3)
memory usage: 41.1+ KB
None
       Cust_ID  Credit_Score
count   945.00        946.00
mean    499.47        626.69
std     289.58        701.30
min

In [33]:
# 1. 去除前後空白 (Trim)
df['Name'] = df['Name'].str.strip()

# 2. 大小寫統一
df['Name'] = df['Name'].str.lower() # 或 .str.upper()

# 3. 取代字元 (去除貨幣符號、逗號)
# 情境：把 '$1,200' 變成 '1200'
df['Income'] = df['Income'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)


In [34]:
# 1. 強制轉數值 (核彈級指令 💣)
# errors='coerce' 是重點：遇到 'Unknown' 或亂碼，直接變成 NaN，不要報錯！
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')

# 2. 轉換後處理 NaN (例如填 0)
df['Income'] = df['Income'].fillna(0)

# 3. 轉成整數 (確認沒小數點後)
df['Income'] = df['Income'].astype(int)

# 4. 時間格式轉換
# 1. 轉成字串 -> 2. 去除前後空白 -> 3. 把 '/' 全部換成 '-'
df['Join_Date_Fixed'] = df['Join_Date'].astype(str).str.strip().str.replace('/', '-')

# 4. 再轉成日期
df['Join_Date_Clean'] = pd.to_datetime(df['Join_Date_Fixed'], errors='coerce')

# 檢查結果
print(df[['Join_Date', 'Join_Date_Clean']].head())

    Join_Date Join_Date_Clean
0  2024/11/28      2024-11-28
1  2024-09-27      2024-09-27
2  2024-08-31      2024-08-31
3  2024-09-12      2024-09-12
4  2024-03-11      2024-03-11


In [35]:
# --- 缺值處理 (Missing Values) ---
# 檢查每一欄有多少缺值
print(df.isna().sum())

# 策略 1: 直接刪除 (若資料量夠大)
df_clean = df.dropna()

# 策略 2: 填補
df['Income'] = df['Income'].fillna(df['Income'].mean())       # 數值填平均
df['Name'] = df['Name'].fillna('Unknown')            # 類別填 'Unknown'

# --- 異常值處理 (Outliers) ---
# 情境：處理 9999 (系統預設錯誤碼)
# 方法 A: 蓋掉變成 NaN
df.loc[df['Credit_Score'] == 9999, 'Credit_Score'] = np.nan 

# 方法 B: 截斷 (Capping) - 超過 850 的都當作 850
df.loc[df['Credit_Score'] > 850, 'Credit_Score'] = 850

Cust_ID            105
Name               108
Income               0
Join_Date          105
Credit_Score       104
Join_Date_Fixed      0
Join_Date_Clean    105
dtype: int64


In [36]:
# 條件篩選
# 例如：選出 Age > 30 且 City 是 'Taipei'
# mask = (df['age'] > 30) & (df['city'] == 'Taipei')
# filtered_df = df[mask].copy() # 加上 .copy() 避免警告

# 去除重複值
df = df.drop_duplicates()

# 欄位重新命名
# df = df.rename(columns={'Old_Name': 'New_Name'})

In [37]:
df['month'] = df['Join_Date_Clean'].dt.month       # 取出月份
df['weekday'] = df['Join_Date_Clean'].dt.day_name()# 取出星期幾
df['year'] = df['Join_Date_Clean'].dt.year

In [38]:
# 重置索引 (非常重要！特別是在 dropna 或篩選後)
df = df.reset_index(drop=True)

# 存檔
# df.to_csv('data_clean.csv', index=False, encoding='utf-8-sig')

df

,Cust_ID,Name,Income,Join_Date,Credit_Score,Join_Date_Fixed,Join_Date_Clean,month,weekday,year
0,1.00,user_1,85093,2024/11/28,718.00,2024-11-28,2024-11-28,11.00,Thursday,2024.00
1,2.00,user_2,108773,2024-09-27,479.00,2024-09-27,2024-09-27,9.00,Friday,2024.00
2,3.00,Unknown,120050,2024-08-31,348.00,2024-08-31,2024-08-31,8.00,Saturday,2024.00
3,4.00,user_4,33807,2024-09-12,405.00,2024-09-12,2024-09-12,9.00,Thursday,2024.00
4,5.00,user_5,112937,2024-03-11,NaN,2024-03-11,2024-03-11,3.00,Monday,2024.00
...,...,...,...,...,...,...,...,...,...,...
995,996.00,user_996,76251,NaN,811.00,nan,NaT,NaN,NaN,NaN
996,NaN,user_997,40761,2024-02-21,603.00,2024-02-21,2024-02-21,2.00,Wednesday,2024.00
997,998.00,user_998,0,2024/04/08,506.00,2024-04-08,2024-04-08,4.00,Monday,2024.00
998,999.00,user_999,45407,2024-05-20,459.00,2024-05-20,2024-05-20,5.00,Monday,2024.00


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Cust_ID          900 non-null    float64       
 1   Name             1000 non-null   object        
 2   Income           1000 non-null   int64         
 3   Join_Date        900 non-null    object        
 4   Credit_Score     896 non-null    float64       
 5   Join_Date_Fixed  1000 non-null   object        
 6   Join_Date_Clean  900 non-null    datetime64[ns]
 7   month            900 non-null    float64       
 8   weekday          900 non-null    object        
 9   year             900 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 78.3+ KB


In [41]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 676 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Cust_ID          676 non-null    float64       
 1   Name             676 non-null    object        
 2   Income           676 non-null    int64         
 3   Join_Date        676 non-null    object        
 4   Credit_Score     676 non-null    float64       
 5   Join_Date_Fixed  676 non-null    object        
 6   Join_Date_Clean  676 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 42.2+ KB
